In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.regularizers import l2
import yfinance as yf

In [ ]:
def fetch_data(ticker, start_date, end_date):
  df= yf.download(ticker, start_date, end_date)
  return df

def preprocess_data(df):
  df= df['Close'].values.reshape(-1,1)
  scaler= MinMaxScaler(feature_range=(0, 1))
  df = scaler.fit_transform(df)
  return df, scaler

df= fetch_data('TSLA','2012-01-01', '2024-01-01')
df, scaler = preprocess_data(df)

[*********************100%%**********************]  1 of 1 completed


In [ ]:
def prepare_data(df, look_back):
    X, Y = [], []
    for i in range(look_back, len(df)):
        X.append(df[i-look_back:i])
        Y.append(df[i, 0])
    X, Y = np.array(X), np.array(Y)
    return X, Y

train_size = int(len(df) * 0.80)
train, test = df[0:train_size,:], df[train_size:len(df),:]
look_back = 60
X_train, Y_train = prepare_data(train, look_back)
X_test, Y_test = prepare_data(test, look_back)

In [ ]:
def build_model():
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, kernel_regularizer=l2(0.01), input_shape=(X_train.shape[1], 1)))
    model.add(Dropout(0.2))
    model.add(LSTM(units=50, kernel_regularizer=l2(0.01)))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

model = build_model()

In [ ]:
model.fit(X_train, Y_train, epochs=50, batch_size=32, verbose=2)

model.save("saved_model.h5")
import joblib
joblib.dump(scaler, 'scaler.pkl')

Epoch 1/50
74/74 - 4s - loss: 0.0027 - 4s/epoch - 49ms/step
Epoch 2/50
74/74 - 5s - loss: 0.0019 - 5s/epoch - 67ms/step
Epoch 3/50
74/74 - 4s - loss: 0.0016 - 4s/epoch - 51ms/step
Epoch 4/50
74/74 - 4s - loss: 0.0017 - 4s/epoch - 49ms/step
Epoch 5/50
74/74 - 4s - loss: 0.0018 - 4s/epoch - 54ms/step
Epoch 6/50
74/74 - 5s - loss: 0.0015 - 5s/epoch - 63ms/step
Epoch 7/50
74/74 - 4s - loss: 0.0019 - 4s/epoch - 49ms/step
Epoch 8/50
74/74 - 4s - loss: 0.0014 - 4s/epoch - 48ms/step
Epoch 9/50
74/74 - 5s - loss: 0.0014 - 5s/epoch - 69ms/step
Epoch 10/50
74/74 - 4s - loss: 0.0013 - 4s/epoch - 48ms/step
Epoch 11/50
74/74 - 4s - loss: 0.0016 - 4s/epoch - 48ms/step
Epoch 12/50
74/74 - 4s - loss: 0.0015 - 4s/epoch - 60ms/step
Epoch 13/50
74/74 - 4s - loss: 0.0014 - 4s/epoch - 57ms/step
Epoch 14/50
74/74 - 4s - loss: 0.0017 - 4s/epoch - 51ms/step
Epoch 15/50
74/74 - 4s - loss: 0.0013 - 4s/epoch - 51ms/step
Epoch 16/50
74/74 - 5s - loss: 0.0013 - 5s/epoch - 68ms/step
Epoch 17/50
74/74 - 4s - loss: 0.

['scaler.pkl']